In [73]:
import pandas as pd
import numpy as np
import sys
import os
import datetime as dt
sys.path.append('./Others/Data_Science/Project')

In [ ]:
Temp2023 = pd.read_csv('Temp2023.csv', encoding='cp949')
Temp2023 = Temp2023[['일시','기온(°C)','강수량(mm)']]

In [ ]:
# nan을 0으로 채우기
Temp2023['강수량(mm)'] = Temp2023['강수량(mm)'].fillna(0)

/tmp/ipykernel_14121/2381164496.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Temp2023['강수량(mm)'] = Temp2023['강수량(mm)'].fillna(0)


In [ ]:
# 일시에서 시간을 따로 추출하여 행으로 추가함
# 일시는 날짜 성분만 남김
Temp2023['일시'] = pd.to_datetime(Temp2023['일시'])
hours_to_exclude = [1, 2, 3, 4]
Temp2023 = Temp2023[~Temp2023['일시'].dt.hour.isin(hours_to_exclude)]
Temp2023['시간'] = Temp2023['일시'].dt.hour
Temp2023['일시'] = Temp2023['일시'].dt.date


In [80]:
Temp2023 = Temp2023.drop(Temp2023.index[0])

# 시간이 0인 행들의 일시에서 하루 빼기
# 먼저 일시를 다시 datetime으로 변환 (현재 date 타입이므로)
Temp2023['일시'] = pd.to_datetime(Temp2023['일시'])

# 시간이 0인 행들의 일시에서 1일 빼기
mask = Temp2023['시간'] == 0
Temp2023.loc[mask, '일시'] = Temp2023.loc[mask, '일시'] - pd.Timedelta(days=1)

# 필요하다면 다시 date 타입으로 변환
Temp2023['일시'] = Temp2023['일시'].dt.date

# 인덱스 재설정 (선택사항)
Temp2023 = Temp2023.reset_index(drop=True)

In [ ]:
sub2023 = pd.read_csv('sub2023.csv', encoding='cp949')

In [ ]:
# 시청 먼저 해보기기
sub2023_cityhall = sub2023[(sub2023['역명']=='시청')]
sub2023_cityhall['수송일자'] = pd.to_datetime(sub2023_cityhall['수송일자'])

/tmp/ipykernel_14121/794108682.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub2023_cityhall['수송일자'] = pd.to_datetime(sub2023_cityhall['수송일자'])


In [82]:
# 새로운 데이터프레임 생성
# 시간을 새로운 column으으로 추가할 것임
passenger = sub2023_cityhall.iloc[:,6:].values.flatten()
hour = np.arange(0,len(passenger)) % 20 + 5
hour[hour == 24] = 0
sub2023df = pd.DataFrame({
    '일자': sub2023_cityhall['수송일자'].values.repeat(20),
    '시간': hour,
    '승하차구분' : sub2023_cityhall['승하차구분'].values.repeat(20),
    '호선' : sub2023_cityhall['호선'].values.repeat(20),
    '승차인원': passenger
})

In [87]:
# sub2023df의 일자를 date 타입으로 변환 (비교를 위해)
sub2023df['일자_date'] = sub2023df['일자'].dt.date

# 일자와 시간을 기준으로 left join
sub2023dfarr = sub2023df.merge(
    Temp2023,
    left_on=['일자_date', '시간'],
    right_on=['일시', '시간'],
    how='left'
)
sub2023dfarr = sub2023dfarr.drop(columns=['일자_date', '일시'])